In [ ]:
%%capture
!pip install -r ../requirements.txt

In [40]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Add it to sys.path if not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [41]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [42]:
from notion_client import Client
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from twilio.rest import Client as TwilioClient
from typing import TypedDict
import os
from tools.notion_fetch_agent_tools import fetch_latest_notion_journaling_entry
from typing import TypedDict
from langgraph.graph import END, StateGraph

In [43]:
# TEMPLATE ENV VARIABLES
GRAPH_ENTRY_QUERY = "Send me bible scripture that will help me with how i want to grow from my latest journaling in Notion."

In [44]:
llm = ChatOpenAI(model="gpt-4o")

In [156]:
class State(TypedDict):
    query: str
    notion_data_action: str # TEMPLATE VAR
    notion_next_node_instructions: str # TEMPLATE VAR
    notion_journal_growth_summary: str # TEMPLATE VAR
    message: str # TEMPLATE VAR

## Agent 1 - Notion

In each agent (agents/), we can have the this code as attributes/methods

In [126]:
from langchain.agents import initialize_agent

In [127]:
tools = [fetch_latest_notion_journaling_entry]

In [128]:
notion_prompt_template = PromptTemplate(
    input_variables=["notion_data_action"],
    template="You are an assistant responsible for simply making a call to the notion client API, and then doing the following instructions on the returned data: {notion_data_action}"
)

In [129]:
notion_agent = initialize_agent(
    tools=tools,
    llm=llm,
    handle_parsing_errors=True
)

In [56]:
result = notion_agent.invoke(notion_prompt_template.format(notion_data_action="From this journaling, extract out the ways the user desires to grow."))

In [57]:
result

{'input': '\n    You are an assistant responsible for simply making a call to the notion client API,\n    and then doing the following instructions on the returned data:\n    From this journaling, extract out the ways the user desires to grow.\n    ',
 'output': 'The user desires to grow spiritually, as a leader, by embracing intrinsic motivation, strengthening relationships, maintaining joy and gratitude, and through personal accountability and a growth mindset.'}

In [157]:
def notion_node(state: State) -> State:
    notion_data_action = state.get("notion_data_action")
    
    prompt = notion_prompt_template.format(notion_data_action=notion_data_action)
    result = notion_agent.invoke(prompt)
    return {"notion_journal_growth_summary": result.get('output')}

## Agent 2 - Scripture (generation)

In [97]:
generation_agent_instructions = """
You are a generalist assistant reponsible for performing a simple query. The 
instructions are:

{instructions}
"""

In [98]:
generation_agent_prompt_template = PromptTemplate(
    # TEMPLATE VAR (output of notion node or other things)
    input_variables=["instructions"],
    # TEMPLATE VAR
    template=generation_agent_instructions
)

In [82]:
generation_agent_llm_chain = LLMChain(llm=llm, prompt=generation_agent_prompt_template)

In [77]:
# Testing
x = "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: {notion_journal_growth_summary}"
x = generation_agent_prompt_template.format(instructions=x).format(notion_journal_growth_summary="The user desires to grow spiritually, as a leader, by embracing intrinsic motivation, strengthening relationships, maintaining joy and gratitude, and through personal accountability and a growth mindset.")
generation_agent_llm_chain.run(x)

'Philippians 4:13 (NIV): "I can do all this through him who gives me strength." \n\nThis verse emphasizes intrinsic motivation, spiritual strength, and the empowerment needed for growth in various personal areas, including leadership and relationships.'

In [161]:
def generation_node(state: State) -> State:
    # Output from previous node
    # TEMPLATE VAR
    notion_journal_growth_summary = state.get("notion_journal_growth_summary")
    # TEMPLATE VAR
    notion_next_node_instructions = state.get("notion_next_node_instructions") + notion_journal_growth_summary
    
    prompt = generation_agent_prompt_template.format(instructions=notion_next_node_instructions)
    output = generation_agent_llm_chain.run(prompt)

    return {"message": output}

## Agent 3 - Twilio

In [159]:
from langchain.tools import tool

TO_NUM = "xyz" # TEMPLATE VAR
FROM_NUM = "xyz" # ENV VAR for the job (set on Twilio console)

@tool
def text_user(msg: str):
    """Tool to text the user via Twilio."""
    # account_sid = os.getenv("TWILIO_ACCOUNT_SID") # TODO: get these
    # auth_token = os.getenv("TWILIO_AUTH_TOKEN")
    # from_number = os.getenv("TWILIO_FROM_NUMBER")
    # client = Client(account_sid, auth_token)
    # sms = client.messages.create(
    #     body=msg,
    #     from_=from_num,
    #     to=to_num
    # )
    #return sms.sid
    print(f"Sent {msg}")

In [160]:
twilio_prompt_template = PromptTemplate(
    # TEMPLATE VAR (output of notion node or other things)
    input_variables=["message"],
    # TEMPLATE VAR
    template="Your sole purpose is to send this text message to the user: \"{message}\""
)

In [162]:
tools = [text_user]
twilio_agent = initialize_agent(
    tools=tools,
    llm=llm,
    handle_parsing_errors=True
)

In [163]:
def twilio_node(state: State) -> State:
    output = twilio_agent.invoke(twilio_prompt_template.format(message=state.get("message")))
    return {}

## Graph

In [164]:
# Initialize the graph with the state schema
graph = StateGraph(State)

# Add nodes
# TEMPLATE VAR - We can loop through the nodes in the actual app
graph.add_node("notion_node", notion_node)
graph.add_node("generation_node", generation_node)
graph.add_node("twilio_node", twilio_node)

# Set the entry point
# TEMPLATE VAR
graph.set_entry_point("notion_node")

# Define sequential edges
# TEMPLATE VAR -Also loop through this
graph.add_edge("notion_node", "generation_node")

graph.add_edge("generation_node", END)
graph.add_edge("generation_node", "twilio_node")
graph.add_edge("twilio_node", END)

compiled_graph = graph.compile()

In [165]:
# This initial state is going to contain the injected variables from the orchestrator
initial_state = {
    "query": GRAPH_ENTRY_QUERY,
    "notion_data_action": "From this journaling, extract out the primary way the user desires to grow in his/her life.",
    "notion_next_node_instructions": "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: "
}
try:
    result = compiled_graph.invoke(initial_state)
    print("Final state:", result)
except Exception as e:
    print(f"Graph-level error: {e}")

Sent A fitting Bible verse that aligns with the user's desire for personal growth through intrinsic motivation, servant leadership, purposeful use of gifts, and fostering community driven by faith and vision is: 1 Peter 4:10 (NIV): 'Each of you should use whatever gift you have received to serve others, as faithful stewards of God's grace in its various forms.'
Final state: {'query': 'Send me bible scripture that will help me with how i want to grow from my latest journaling in Notion.', 'notion_data_action': 'From this journaling, extract out the primary way the user desires to grow in his/her life.', 'notion_next_node_instructions': "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: ", 'notion_journal_growth_summary': 'The user desires to grow through intrinsic motivation, embracing servant leadership, using their gifts purposefully, and fostering community, driven by their faith and vision.', 'mess